In [57]:
import os

import numpy as np
import scipy.misc
from sklearn.cross_validation import *
from sklearn.metrics import *
from sklearn.preprocessing import *
import tensorflow as tf
from PIL import Image

In [58]:
RED = 0
GREEN = 1
BLUE = 2

In [59]:
def determine_color_from_filename(filename):
    if 'blue' in filename.lower():
        return BLUE
    if 'green' in filename.lower():
        return GREEN
    return RED

In [60]:
def scale_unit_feature(X):
    unit_scaler = MinMaxScaler((0, 1))
    original_size = X.shape
    X = np.reshape(X, (X.shape[0], np.prod(X.shape[1:])))
    X = unit_scaler.fit_transform(X)
    X = np.reshape(X, original_size)
    return X

In [61]:
def convert_to_one_hot(y):
    return OneHotEncoder(3, sparse=False).fit_transform(y).astype('int32')

In [62]:
raw_X = []
raw_y = []
for folder_path, folders, filenames in os.walk('data/input'):
    for filename in filenames:
        filepath = os.path.join(folder_path, filename)
        image = Image.open(filepath).convert('RGB')
        image = np.array(image)
        raw_X.append(image)
        raw_y.append(determine_color_from_filename(filename))

X = np.array(raw_X, dtype='float32')
y = np.array(raw_y, dtype='int32')

In [41]:
print('X.shape: ', X.shape)
print('y.shape: ', y.shape)

X.shape:  (470, 64, 64, 3)
y.shape:  (470,)


In [42]:
# X = scale_unit_feature(X)
# y = convert_to_one_hot(y)

In [43]:
print('X.shape: ', X.shape)
print('y.shape: ', y.shape)

X.shape:  (470, 64, 64, 3)
y.shape:  (470,)


In [64]:
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size = 0.2)

In [74]:
print('X_train.shape: ', X_train.shape)
print('X_cv.shape: ', X_cv.shape)
print('y_train.shape: ', y_train.shape)
print('y_cv.shape: ', y_cv.shape)

X_train.shape:  (376, 64, 64, 3)
X_cv.shape:  (94, 64, 64, 3)
y_train.shape:  (376,)
y_cv.shape:  (94,)


Begin Tensorflow Code

In [75]:
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)

In [76]:
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[10],
                                            n_classes=3,
                                            model_dir="lab_model.dat")


Explicitly set `enable_centered_bias` to 'True' if you want to keep existing behaviour.


In [77]:
classifier.fit(X_train, y_train, batch_size=100, steps=2000)

Estimator(params={'activation_fn': <function relu at 0x7f7faa2b2378>, 'enable_centered_bias': True, 'n_classes': 3, 'gradient_clip_norm': None, 'num_ps_replicas': 0, 'weight_column_name': None, 'dropout': None, 'hidden_units': [10], 'optimizer': 'Adagrad', 'feature_columns': [_RealValuedColumn(column_name='', dimension=12288, default_value=None, dtype=tf.float32, normalizer=None)]})

In [78]:
y_cv_pred = classifier.predict(X_cv)

Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.


In [80]:
print(y_cv_pred)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
